In [1]:
import sys
sys.path.insert(0, r'P:\pet ML')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
from data_preprocessing import preprocessing
from work_with_db import work_with_db
import sqlite3 as sql

In [2]:
BASE_DIR = 'P:/pet ML'

In [3]:
def process():
    db = work_with_db()
    prep = preprocessing()
    df = pd.DataFrame(db.action_with_db(action='SELECT', columns='*'))
    df = df.dropna()
    prep.train(data_tr=df)

In [4]:
process()

In [5]:
def norm():
    db = work_with_db()
    prep = preprocessing()
    data = pd.DataFrame(db.action_with_db(action='SELECT', columns='*'))
    data = data.dropna()
    data_x, data_y = prep.normalization(df=data)
    return data_x, data_y

In [9]:
x,y = norm()

In [ ]:
def to_tensor():
    prep = preprocessing()
    prep.to_tenser(X=x, y=y)
    